In [ ]:
# Análisis Integral de Estudiantes (Dataset 02)

Este cuaderno integra tres enfoques de clustering para segmentar el dataset de estudiantes basándose en su rendimiento académico y perfil personal.

1. **K-Means**: Segmentación por niveles de rendimiento académico.
2. **DBSCAN**: Identificación de grupos densos y detección de outliers (ruido).
3. **K-Prototypes**: Clustering para variables mixtas (numéricas y categóricas).
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 1. Carga y Exploración Inicial de Datos
Cargamos el archivo `Estudiantes.csv` y revisamos su estructura básica.
</VSCode.Cell>

<VSCode.Cell language="python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from kmodes.kprototypes import KPrototypes
import warnings
warnings.filterwarnings('ignore')

# Cargar datos
df = pd.read_csv('data/Estudiantes.csv')
print(f"Dimensiones del dataset: {df.shape}")
df.head()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 2. Clasificación con K-Means
Implementamos K-Means utilizando las notas de Matemáticas, Lectura y Escritura.
</VSCode.Cell>

<VSCode.Cell language="python">
# Características para clustering
columnas_notas = ['Matematicas', 'Lectura', 'Escritura']
X_kmeans = df[columnas_notas]

# Escalamiento
scaler = StandardScaler()
X_kmeans_scaled = scaler.fit_transform(X_kmeans)

# Aplicación de KMeans
n_clusters_km = 5
kmeans = KMeans(n_clusters=n_clusters_km, init='k-means++', n_init=10, random_state=42)
df['Cluster_KMeans'] = kmeans.fit_predict(X_kmeans_scaled)

print("Distribución de clusters (K-Means):")
print(df['Cluster_KMeans'].value_counts())

# Resumen de promedios por cluster
df.groupby('Cluster_KMeans')[columnas_notas].mean()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 3. Visualización de Regiones de Decisión
Graficamos la separación de clusters en 2D (Lectura vs Escritura) mostrando las fronteras de decisión.
</VSCode.Cell>

<VSCode.Cell language="python">
# Preparar datos 2D
X_2d = df[['Lectura', 'Escritura']].values
km_2d = KMeans(n_clusters=n_clusters_km, random_state=42).fit(X_2d)

# Malla para regiones
h = 0.5
x_min, x_max = X_2d[:, 0].min() - 5, X_2d[:, 0].max() + 5
y_min, y_max = X_2d[:, 1].min() - 5, X_2d[:, 1].max() + 5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

Z = km_2d.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(12, 7))
plt.imshow(Z, interpolation='nearest', extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Pastel2, aspect='auto', origin='lower')

plt.scatter(X_2d[:, 0], X_2d[:, 1], c=km_2d.labels_, s=20, edgecolor='white', alpha=0.6, cmap='viridis')
plt.scatter(km_2d.cluster_centers_[:, 0], km_2d.cluster_centers_[:, 1], s=200, marker='X', c='red', label='Centroides')

plt.title('Regiones de Decisión: Lectura vs Escritura (K-Means)')
plt.xlabel('Puntaje Lectura')
plt.ylabel('Puntaje Escritura')
plt.legend()
plt.show()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 4. Implementación de Clustering con DBSCAN
Utilizamos DBSCAN para encontrar grupos de alta densidad y detectar estudiantes con puntajes atípicos (outliers).
</VSCode.Cell>

<VSCode.Cell language="python">
# Usamos los mismos datos escalados
dbscan = DBSCAN(eps=0.5, min_samples=5)
df['Cluster_DBSCAN'] = dbscan.fit_predict(X_kmeans_scaled)

n_noise = list(df['Cluster_DBSCAN']).count(-1)
print(f"Clusters encontrados: {len(set(df['Cluster_DBSCAN'])) - (1 if -1 in df['Cluster_DBSCAN'] else 0)}")
print(f"Puntos de ruido detectados: {n_noise}")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 5. Visualización de Resultados de DBSCAN
Mostramos cómo DBSCAN separa el grupo principal de los casos aislados o atípicos.
</VSCode.Cell>

<VSCode.Cell language="python">
plt.figure(figsize=(15, 5))

# Matematicas vs Lectura
plt.subplot(1, 2, 1)
sns.scatterplot(data=df[df['Cluster_DBSCAN'] != -1], x='Matematicas', y='Lectura', hue='Cluster_DBSCAN', palette='viridis', alpha=0.5)
sns.scatterplot(data=df[df['Cluster_DBSCAN'] == -1], x='Matematicas', y='Lectura', color='red', marker='x', s=100, label='Ruido')
plt.title('DBSCAN: Matematicas vs Lectura')

# Lectura vs Escritura
plt.subplot(1, 2, 2)
sns.scatterplot(data=df[df['Cluster_DBSCAN'] != -1], x='Lectura', y='Escritura', hue='Cluster_DBSCAN', palette='viridis', alpha=0.5)
sns.scatterplot(data=df[df['Cluster_DBSCAN'] == -1], x='Lectura', y='Escritura', color='red', marker='x', s=100, label='Ruido')
plt.title('DBSCAN: Lectura vs Escritura')

plt.tight_layout()
plt.show()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 6. Clustering para Datos Mixtos con K-Prototypes
Este algoritmo agrupa estudiantes considerando tanto sus notas (numéricas) como sus perfiles demográficos (categóricos).
</VSCode.Cell>

<VSCode.Cell language="python">
# Preparar datos para K-Prototypes
categorical_idx = [0, 1, 2, 3] # Genero, Etnia, Nivel educativo, Examen preparacion
cols_kproto = ['Genero', 'Etnia', 'Nivel educativo de los padres', 'Examen de preparacion', 
               'Matematicas', 'Lectura', 'Escritura']

df_transformed = df[cols_kproto].copy()
df_transformed[columnas_notas] = scaler.fit_transform(df[columnas_notas])

# Ejecutar K-Prototypes
kproto = KPrototypes(n_clusters=4, init='Cao', verbose=0, random_state=42)
df['Cluster_KProto'] = kproto.fit_predict(df_transformed.values, categorical=categorical_idx)

print("Distribución de clusters (K-Prototypes):")
print(df['Cluster_KProto'].value_counts())
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 7. Visualización de Resultados de K-Prototypes
Analizamos la composición de los clusters basándonos en variables categóricas y notas.
</VSCode.Cell>

<VSCode.Cell language="python">
plt.figure(figsize=(15, 6))

# Boxplot de Notas
plt.subplot(1, 2, 1)
df_melted = df.melt(id_vars=['Cluster_KProto'], value_vars=columnas_notas, var_name='Materia', value_name='Nota')
sns.boxplot(data=df_melted, x='Cluster_KProto', y='Nota', hue='Materia')
plt.title('Rendimiento Académico por Cluster (K-Proto)')

# Composición Educativa de los Padres
plt.subplot(1, 2, 2)
sns.countplot(data=df, x='Cluster_KProto', hue='Nivel educativo de los padres')
plt.title('Nivel Educativo de los Padres por Cluster')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')

plt.tight_layout()
plt.show()
</VSCode.Cell>
